The dataset can be downloaded here:

https://www.kaggle.com/c/dogs-vs-cats/data

In [3]:
import os
import glob

import numpy as np

In [2]:
INP_TRAIN_DIR = "data/download/train"
INP_TEST_DIR = "data/download/test1"

In [7]:
train_dog_files = glob.glob(os.path.join(INP_TRAIN_DIR, "dog*.jpg"))
train_cat_files = glob.glob(os.path.join(INP_TRAIN_DIR, "cat*.jpg"))

np.random.seed(1)
np.random.shuffle(train_dog_files)
np.random.shuffle(train_cat_files)

len(train_dog_files), len(train_cat_files)

(12500, 12500)